###   Importing necessary library

In [1]:
import os

import tensorflow as tf
import tensorflow as gfile

from typing import List
import numpy as np
from PIL import Image
import time

In [2]:
from six import iteritems, string_types

In [3]:
FACE_PIC_SIZE = 160

EMBEDDING_SIZE = 512

PRETREINED_MODEL_DIR = os.path.join(os.getcwd(),'pretrained_models')

UNKNOWN_CLASS = "unknown"

In [4]:
PRETREINED_MODEL_DIR

'F:\\MLTUT\\pretrained_models'

In [5]:
import cv2

### Implementation of Layers

In [6]:
padding = ['SAME','VALID']

In [7]:
def load(data_path, session, ignore_missing=False):
    """Load network weights.
    data_path: The path to the numpy-serialized network weights
    session: The current TensorFlow session
    ignore_missing: If true, serialized weights for missing layers are ignored.
    """
    data_dict = np.load(data_path, encoding='latin1').item()  # pylint: disable=no-member

    for op_name in data_dict:
        with tf.variable_scope(op_name, reuse=True):
            for param_name, data in iteritems(data_dict[op_name]):
                try:
                    var = tf.get_variable(param_name)
                    session.run(var.assign(data))
                except ValueError:
                    if not ignore_missing:
                        raise

In [8]:
def conv(inpt, k_h, k_w, c_o, s_h, s_w, name, relu=True, padding=padding[0], group=1, biased=True):
    """
    @inpt: input data to convolve
    @k_h: kernel height  
    @k_w: kernel width
    @c_o: number of filter
    @s_h: strides weight
    @s_w: strides height 
    @name: layer name
    @relu: non linearity RELU
    @padding: filter padding 
    """
# Get the number of channels in the input
    c_i = int(inpt.get_shape()[-1])

    # Convolution for a given input and kernel
    with tf.variable_scope(name) as scope:
        kernel = tf.get_variable('weights', shape=[k_h, k_w, c_i // group, c_o], trainable=True)
        output = tf.nn.conv2d(inpt, kernel, [1, s_h, s_w, 1], padding=padding)
        # Add the biases
        if biased:
            biases = tf.get_variable('biases', [c_o])
            output = tf.nn.bias_add(output, biases)
        if relu:
            # ReLU non-linearity
            output = tf.nn.relu(output, name=scope.name)
        return output

In [9]:
def prelu(inpt, name):
    with tf.variable_scope(name):
        i = int(inpt.get_shape()[-1])
        alpha = tf.get_variable('alpha', shape=(i,))
        output = tf.nn.relu(inpt) + tf.multiply(alpha, -tf.nn.relu(-inpt))
    return output

In [10]:
def max_pool(inpt, k_h, k_w, s_h, s_w, name, padding=padding[0]):
    return tf.nn.max_pool(inpt, ksize=[1, k_h, k_w, 1],strides=[1, s_h, s_w, 1], padding=padding, name=name)

In [11]:
def fully_connected(inpt, num_out, name, relu=True):
    with tf.variable_scope(name):
        input_shape = inpt.get_shape()
        if input_shape.ndims == 4:
            # since the input is spatial.we have to Vectorize it first.
            dim = 1
            for d in input_shape[1:].as_list():
                dim *= int(d)
            feed_in = tf.reshape(inpt, [-1, dim])
        else:
            feed_in, dim = (inpt, input_shape[-1].value)
        weights = tf.get_variable('weights', shape=[dim, num_out])
        biases = tf.get_variable('biases', [num_out])
        op = tf.nn.relu_layer if relu else tf.nn.xw_plus_b
        fc = op(feed_in, weights, biases, name=name)
        return fc

In [12]:
def softmax(target, axis, name=None):
    max_axis = tf.reduce_max(target, axis, keepdims=True)
    target_exp = tf.exp(target - max_axis)
    normalize = tf.reduce_sum(target_exp, axis, keepdims=True)
    softmax = tf.div(target_exp, normalize, name)
    return softmax

## Implementation of MTCNN

### P-Net ( Proposal - Network )

In [13]:
def pNet(data):
    value = conv(data, 3, 3, 10, 1, 1, padding='VALID', relu=False, name='conv1')
    value = prelu(value, name='PReLU1')
    value = max_pool(value, 2, 2, 2, 2, name='pool1')
    value = conv(value ,3, 3, 16, 1, 1, padding='VALID', relu=False, name='conv2')
    value = prelu(value ,name='PReLU2')
    value = conv(value ,3, 3, 32, 1, 1, padding='VALID', relu=False, name='conv3')
    value = prelu(value ,name='PReLU3')
#     Face detection classfication probablity output
    val_conv4_1 = conv(value ,1, 1, 2, 1, 1, relu=False, name='conv4-1')
    classify = softmax(val_conv4_1 ,3, name='prob1')
#     bounding box regression
    bbr = conv(value ,1, 1, 4, 1, 1, relu=False, name='conv4-2')
    
    return(classify, bbr)

In [14]:
def rNet(data):
    value = conv(data, 3, 3, 28, 1, 1, padding='VALID', relu=False, name='conv1')
    value = prelu(value, name='prelu1')
    value = max_pool(value, 3, 3, 2, 2, name='pool1')
    value = conv(value, 3, 3, 48, 1, 1, padding='VALID', relu=False, name='conv2')
    value = prelu(value, name='prelu2')
    value = max_pool(value, 3, 3, 2, 2, padding='VALID', name='pool2')
    value = conv(value, 2, 2, 64, 1, 1, padding='VALID', relu=False, name='conv3')
    value = prelu(value, name='prelu3')
    value = fully_connected(value, 128, relu=False, name='conv4')
    value = prelu(value, name='prelu4')
#     face detection classfication probablity output
    conv5_1 = fully_connected(value, 2, relu=False, name='conv5-1')
    classify = softmax(conv5_1, 1, name='prob1')
#     bounding box regression
    bbr = fully_connected(value,4, relu=False, name='conv5-2')
    
    return (classify, bbr)

In [15]:
def oNet(data):
    value = conv(data, 3, 3, 32, 1, 1, padding='VALID', relu=False, name='conv1')
    value = prelu(value, name='prelu1')
    value = max_pool(value ,3, 3, 2, 2, name='pool1')
    value = conv(value, 3, 3, 64, 1, 1, padding='VALID', relu=False, name='conv2')
    value = prelu(value, name='prelu2')
    value = max_pool(value, 3, 3, 2, 2, padding='VALID', name='pool2')
    value = conv(value, 3, 3, 64, 1, 1, padding='VALID', relu=False, name='conv3')
    value = prelu(value, name='prelu3')
    value = max_pool(value, 2, 2, 2, 2, name='pool3')
    value = conv(value, 2, 2, 128, 1, 1, padding='VALID', relu=False, name='conv4')
    value = prelu(value, name='prelu4')
    value = fully_connected(value, 256, relu=False, name='conv5')
    value = prelu(value, name='prelu5')
#     face detection classification
    conv6_1 = fully_connected(value, 2, relu=False, name='conv6-1')
    classify = softmax(conv6_1, 1, name='prob1')
#     bounding box regression
    bbr = fully_connected(value, 4, relu=False, name='conv6-2')
#     face localization
    floc = fully_connected(value, 10, relu=False, name='conv6-3')
    
    return (classify, bbr, floc)

In [16]:
def create_mtcnn(sess, model_path):
    """
    Input @sess: Current session,
    Input @model_path: path of weights of model to load,
    """
#     Creating variable scope for Pnet
    with tf.variable_scope('pnet'):
        data = tf.placeholder(tf.float32, (None, None, None, 3), 'input')
        pnet = pNet(data)
        load(os.path.join(model_path, 'det1.npy'), sess)
        
#     Creating Variabel scope for Rnet
    with tf.variable_scope('rnet'):
        data = tf.placeholder(tf.float32, (None, 24, 24, 3), 'input')
        rnet = rNet(data)
        load(os.path.join(model_path, 'det2.npy'), sess)
        
#    Creating Variable scope for Onet
    with tf.variable_scope('onet'):
        data = tf.placeholder(tf.float32, (None, 48, 48, 3), 'input')
        onet = oNet(data)
        load(os.path.join(model_path, 'det3.npy'), sess)
        
#         Predicting facial features by cascading P-net, R-net and O-net
    pnet_fun = lambda img: sess.run(('pnet/conv4-2/BiasAdd:0', 'pnet/prob1:0'), feed_dict={'pnet/input:0': img})
    rnet_fun = lambda img: sess.run(('rnet/conv5-2/conv5-2:0', 'rnet/prob1:0'), feed_dict={'rnet/input:0': img})
    onet_fun = lambda img: sess.run(('onet/conv6-2/conv6-2:0', 'onet/conv6-3/conv6-3:0', 'onet/prob1:0'), feed_dict={'onet/input:0': img})
    
    return pnet_fun, rnet_fun, onet_fun

In [17]:
def _setup_mtcnn():
    with tf.Graph().as_default():
        sess = tf.Session()
        with sess.as_default():
            return create_mtcnn(sess, PRETREINED_MODEL_DIR)

In [18]:
pnet, rnet, onet = _setup_mtcnn()

In [19]:
def detect_face(img, minsize, pnet, rnet, onet, threshold, factor):
    """Detects faces in an image, and returns bounding boxes and points for them.
    img: input image
    minsize: minimum faces' size
    pnet, rnet, onet: proposal net, Refinement net, Output net
    threshold: threshold=[th1, th2, th3], th1-3 are three steps's threshold
    factor: the factor used to create a scaling pyramid of face sizes to detect in the image.
    """
    factor_count = 0
    total_boxes = np.empty((0, 9))
    points = np.empty(0)
    h = img.shape[0]
    w = img.shape[1]
    minl = np.amin([h, w])
    m = 12.0 / minsize
    minl = minl * m
    # create scale pyramid
    scales = []
    while minl >= 12:
        scales += [m * np.power(factor, factor_count)]
        minl = minl * factor
        factor_count += 1

    # first stage
    for scale in scales:
        hs = int(np.ceil(h * scale))
        ws = int(np.ceil(w * scale))
        im_data = imresample(img, (hs, ws))
        im_data = (im_data - 127.5) * 0.0078125
        img_x = np.expand_dims(im_data, 0)
        img_y = np.transpose(img_x, (0, 2, 1, 3))
        out = pnet(img_y)
        
        out0 = np.transpose(out[0], (0, 2, 1, 3))
        out1 = np.transpose(out[1], (0, 2, 1, 3))

        boxes, _ = generateBoundingBox(out1[0, :, :, 1].copy(), out0[0, :, :, :].copy(), scale, threshold[0])

        # inter-scale nms
        pick = nms(boxes.copy(), 0.5, 'Union')
        if boxes.size > 0 and pick.size > 0:
            boxes = boxes[pick, :]
            total_boxes = np.append(total_boxes, boxes, axis=0)

    numbox = total_boxes.shape[0]
    if numbox > 0:
        pick = nms(total_boxes.copy(), 0.7, 'Union')
        total_boxes = total_boxes[pick, :]
        regw = total_boxes[:, 2] - total_boxes[:, 0]
        regh = total_boxes[:, 3] - total_boxes[:, 1]
        qq1 = total_boxes[:, 0] + total_boxes[:, 5] * regw
        qq2 = total_boxes[:, 1] + total_boxes[:, 6] * regh
        qq3 = total_boxes[:, 2] + total_boxes[:, 7] * regw
        qq4 = total_boxes[:, 3] + total_boxes[:, 8] * regh
        total_boxes = np.transpose(np.vstack([qq1, qq2, qq3, qq4, total_boxes[:, 4]]))
        total_boxes = rerec(total_boxes.copy())
        total_boxes[:, 0:4] = np.fix(total_boxes[:, 0:4]).astype(np.int32)
        dy, edy, dx, edx, y, ey, x, ex, tmpw, tmph = pad(total_boxes.copy(), w, h)

    numbox = total_boxes.shape[0]
    if numbox > 0:
        # second stage
        tempimg = np.zeros((24, 24, 3, numbox))
        for k in range(0, numbox):
            tmp = np.zeros((int(tmph[k]), int(tmpw[k]), 3))
            tmp[dy[k] - 1:edy[k], dx[k] - 1:edx[k], :] = img[y[k] - 1:ey[k], x[k] - 1:ex[k], :]
            if tmp.shape[0] > 0 and tmp.shape[1] > 0 or tmp.shape[0] == 0 and tmp.shape[1] == 0:
                tempimg[:, :, :, k] = imresample(tmp, (24, 24))
            else:
                return np.empty()
        tempimg = (tempimg - 127.5) * 0.0078125
        tempimg1 = np.transpose(tempimg, (3, 1, 0, 2))
        out = rnet(tempimg1)
        out0 = np.transpose(out[0])
        out1 = np.transpose(out[1])
        score = out1[1, :]
        ipass = np.where(score > threshold[1])
        total_boxes = np.hstack([total_boxes[ipass[0], 0:4].copy(), np.expand_dims(score[ipass].copy(), 1)])
        mv = out0[:, ipass[0]]
        if total_boxes.shape[0] > 0:
            pick = nms(total_boxes, 0.7, 'Union')
            total_boxes = total_boxes[pick, :]
            total_boxes = bbreg(total_boxes.copy(), np.transpose(mv[:, pick]))
            total_boxes = rerec(total_boxes.copy())

    numbox = total_boxes.shape[0]
    if numbox > 0:
        # third stage
        total_boxes = np.fix(total_boxes).astype(np.int32)
        dy, edy, dx, edx, y, ey, x, ex, tmpw, tmph = pad(total_boxes.copy(), w, h)
        tempimg = np.zeros((48, 48, 3, numbox))
        for k in range(0, numbox):
            tmp = np.zeros((int(tmph[k]), int(tmpw[k]), 3))
            tmp[dy[k] - 1:edy[k], dx[k] - 1:edx[k], :] = img[y[k] - 1:ey[k], x[k] - 1:ex[k], :]
            if tmp.shape[0] > 0 and tmp.shape[1] > 0 or tmp.shape[0] == 0 and tmp.shape[1] == 0:
                tempimg[:, :, :, k] = imresample(tmp, (48, 48))
            else:
                return np.empty()
        tempimg = (tempimg - 127.5) * 0.0078125
        tempimg1 = np.transpose(tempimg, (3, 1, 0, 2))
        out = onet(tempimg1)
        out0 = np.transpose(out[0])
        out1 = np.transpose(out[1])
        out2 = np.transpose(out[2])
        score = out2[1, :]
        points = out1
        ipass = np.where(score > threshold[2])
        points = points[:, ipass[0]]
        total_boxes = np.hstack([total_boxes[ipass[0], 0:4].copy(), np.expand_dims(score[ipass].copy(), 1)])
        mv = out0[:, ipass[0]]

        w = total_boxes[:, 2] - total_boxes[:, 0] + 1
        h = total_boxes[:, 3] - total_boxes[:, 1] + 1
        points[0:5, :] = np.tile(w, (5, 1)) * points[0:5, :] + np.tile(total_boxes[:, 0], (5, 1)) - 1
        points[5:10, :] = np.tile(h, (5, 1)) * points[5:10, :] + np.tile(total_boxes[:, 1], (5, 1)) - 1
        if total_boxes.shape[0] > 0:
            total_boxes = bbreg(total_boxes.copy(), np.transpose(mv))
            pick = nms(total_boxes.copy(), 0.7, 'Min')
            total_boxes = total_boxes[pick, :]
            points = points[:, pick]

    return total_boxes, points

In [20]:
def bbreg(boundingbox, reg):
    """Calibrate bounding boxes"""
    if reg.shape[1] == 1:
        reg = np.reshape(reg, (reg.shape[2], reg.shape[3]))

    w = boundingbox[:, 2] - boundingbox[:, 0] + 1
    h = boundingbox[:, 3] - boundingbox[:, 1] + 1
    b1 = boundingbox[:, 0] + reg[:, 0] * w
    b2 = boundingbox[:, 1] + reg[:, 1] * h
    b3 = boundingbox[:, 2] + reg[:, 2] * w
    b4 = boundingbox[:, 3] + reg[:, 3] * h
    boundingbox[:, 0:4] = np.transpose(np.vstack([b1, b2, b3, b4]))
    return boundingbox

In [21]:
def generateBoundingBox(imap, reg, scale, t):
    """Use heatmap to generate bounding boxes"""
    stride = 2
    cellsize = 12

    imap = np.transpose(imap)
    dx1 = np.transpose(reg[:, :, 0])
    dy1 = np.transpose(reg[:, :, 1])
    dx2 = np.transpose(reg[:, :, 2])
    dy2 = np.transpose(reg[:, :, 3])
    y, x = np.where(imap >= t)
    if y.shape[0] == 1:
        dx1 = np.flipud(dx1)
        dy1 = np.flipud(dy1)
        dx2 = np.flipud(dx2)
        dy2 = np.flipud(dy2)
    score = imap[(y, x)]
    reg = np.transpose(np.vstack([dx1[(y, x)], dy1[(y, x)], dx2[(y, x)], dy2[(y, x)]]))
    if reg.size == 0:
        reg = np.empty((0, 3))
    bb = np.transpose(np.vstack([y, x]))
    q1 = np.fix((stride * bb + 1) / scale)
    q2 = np.fix((stride * bb + cellsize - 1 + 1) / scale)
    boundingbox = np.hstack([q1, q2, np.expand_dims(score, 1), reg])
    return boundingbox, reg

In [22]:
# Non - Maximum supression technique
def nms(boxes, threshold, method):
#     if no face detected
    if boxes.size == 0:
        return np.empty((0, 3))
#     initialize the list of picked indexes
    pick = np.zeros_like(s, dtype=np.int16)
#     grab the coordinates of the bounding boxes
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    s = boxes[:, 4]
#     compute the area of the bounding boxes and sort the bounding
#     boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    I = np.argsort(s)
    counter = 0
#     keep looping while some indexes still remain in the indexes
#     list
    while I.size > 0:
#         grab the last index in the indexes list, add the index
#         value to the list of picked indexes, then initialize
#         the suppression list (i.e. indexes that will be deleted)
#         using the last index
        i = I[-1]
        pick[counter] = i
        counter += 1
        idx = I[0:-1]
#         find the largest (x, y) coordinates for the start of
#         the bounding box and the smallest (x, y) coordinates
#         for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idx])
        yy1 = np.maximum(y1[i], y1[idx])
        xx2 = np.minimum(x2[i], x2[idx])
        yy2 = np.minimum(y2[i], y2[idx])
#         compute the widht and height of bounding box
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
#         compute the ratio of overlap between the computed
#         bounding box and the bounding box in the area list
        inter = w * h
        if method is 'Min':
            o = inter / np.minimum(area[i], area[idx])
        else:
            o = inter / (area[i] + area[idx] - inter)
        I = I[np.where(o <= threshold)]
    pick = pick[0:counter]
    return pick


In [23]:
def pad(total_boxes, w, h):
    """Compute the padding coordinates (pad the bounding boxes to square)"""
    tmpw = (total_boxes[:, 2] - total_boxes[:, 0] + 1).astype(np.int32)
    tmph = (total_boxes[:, 3] - total_boxes[:, 1] + 1).astype(np.int32)
    numbox = total_boxes.shape[0]

    dx = np.ones((numbox), dtype=np.int32)
    dy = np.ones((numbox), dtype=np.int32)
    edx = tmpw.copy().astype(np.int32)
    edy = tmph.copy().astype(np.int32)

    x = total_boxes[:, 0].copy().astype(np.int32)
    y = total_boxes[:, 1].copy().astype(np.int32)
    ex = total_boxes[:, 2].copy().astype(np.int32)
    ey = total_boxes[:, 3].copy().astype(np.int32)

    tmp = np.where(ex > w)
    edx.flat[tmp] = np.expand_dims(-ex[tmp] + w + tmpw[tmp], 1)
    ex[tmp] = w

    tmp = np.where(ey > h)
    edy.flat[tmp] = np.expand_dims(-ey[tmp] + h + tmph[tmp], 1)
    ey[tmp] = h

    tmp = np.where(x < 1)
    dx.flat[tmp] = np.expand_dims(2 - x[tmp], 1)
    x[tmp] = 1

    tmp = np.where(y < 1)
    dy.flat[tmp] = np.expand_dims(2 - y[tmp], 1)
    y[tmp] = 1

    return dy, edy, dx, edx, y, ey, x, ex, tmpw, tmph

In [24]:
def rerec(bboxA):
    """Convert bboxA to square."""
    h = bboxA[:, 3] - bboxA[:, 1]
    w = bboxA[:, 2] - bboxA[:, 0]
    l = np.maximum(w, h)
    bboxA[:, 0] = bboxA[:, 0] + w * 0.5 - l * 0.5
    bboxA[:, 1] = bboxA[:, 1] + h * 0.5 - l * 0.5
    bboxA[:, 2:4] = bboxA[:, 0:2] + np.transpose(np.tile(l, (2, 1)))
    return bboxA


def imresample(img, sz):
    im_data = cv2.resize(img, (sz[1], sz[0]), interpolation=cv2.INTER_AREA)  # @UndefinedVariable
    return im_data

In [25]:
class Face:
    # face bounding boxes
    def __init__(self, x, y, w, h, confidence, img):
        self.img = img
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.confidence = confidence

        self.predict_proba = None
        self.predict_candidates = None
        self.predict_candidates_proba = None

    def data(self):
        return { k:v for k, v in self.__dict__.items() if k != 'img'}

In [26]:
def img_to_np(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [27]:
def get_faces(image, threshold=0.5, minsize=20):
    img = img_to_np(image)
    # face detection parameters
    threshold = [0.6, 0.7, 0.7]  # three steps's threshold
    factor = 0.709  # scale factor
    faces = []

    bounding_boxes, _ = detect_face(img, minsize, pnet, rnet, onet,
                                                      threshold, factor)
    for bb in bounding_boxes:
        img = image.crop(bb[:4])
        bb[2:4] -= bb[:2]
        faces.append(Face(*bb, img))

    return faces

In [28]:
cam = cv2.VideoCapture(0)

cv2.namedWindow("test")

img_counter = 0

while True:
    ret, frame = cam.read()
    frame = cv2.flip(frame, 1)
    cv2.imshow("test", frame)
    if not ret:
        break
    k = cv2.waitKey(1)

    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        im = Image.fromarray(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
        faces = get_faces(im)
        for face in faces:
            cv2.rectangle(frame, (int(face.x), int(face.y)), (int(face.x+face.w), int(face.y+face.h)), (0,155,255), 2)
            
        cv2.imshow('frame',frame)

cam.release()

cv2.destroyAllWindows()

Escape hit, closing...
